# fill out age sex splitting distribution
need to make sure all new locations are in there

- every location has the same years
- should be square
- all the new subnational locations have a parent location in the old distribution



In [ ]:
import db_queries

In [ ]:
def check_missing_locations(df, new_location_ids):
    missing_locations = list(set(new_location_ids) - set(df.location_id.unique()))
    if missing_locations:
        print("There are missing locations")
        print(missing_locations)
    return missing_locations

In [ ]:
new_location_metadata = db_queries.get_location_metadata(gbd_round_id=6, location_set_id=82)
old_location_metadata = db_queries.get_location_metadata(gbd_round_id=5, location_set_id=82)

new_location_ids = list(set(new_location_metadata.location_id) - set(old_location_metadata.location_id))

input_distribution_file = ("")

old_distribution = pd.read_csv(input_distribution_file)

In [ ]:
check_missing_locations(old_distribution, new_location_ids)

In [ ]:
new_locations_only_metadata = new_location_metadata[new_location_metadata.location_id.isin(new_location_ids)]

new_parent_location_ids = new_locations_only_metadata[new_locations_only_metadata.level == 4].parent_id.unique()

In [ ]:
print(old_distribution[old_distribution.location_id.isin(new_parent_location_ids)].location_id.unique())
print(new_parent_location_ids)

1. ~~I want to make a dictionary that goes parent_id : subnational id~~
2. Then for a parent_id, get the subset of the old distribution that has that parent id
3. then take that subset, and replace the parent_id location_id with one of the child location_ids
4. repeat for each child location
5. append all child location dataframes together.
6. append that to the distribution file

parent-to-children dict allows me to run this for each parent id


In [ ]:
new_parent_location_ids

In [ ]:
new_parent_location_ids

In [ ]:
def make_weights_for_new_subnationals(distribution_df, parent_id):
    
    # subset distribution df to just the dist for the parent_id
    parent_distribution_df = distribution_df[distribution_df.location_id == parent_id].copy()
    assert parent_distribution_df.shape[0] > 0, "NO DATA FOR THIS LOCATION: {}".foramt(parent_id)
    
    # get location_metadata for the children locations of the parent and test them
    children_location_metadata = new_location_metadata[
        new_location_metadata.path_to_top_parent.str.contains(",{},".format(parent_id))
    ].copy()
    assert (children_location_metadata.level == 4).all()
    # get list of children location ids
    children_location_ids = children_location_metadata.location_id.unique().tolist()
    
    # could do a location_id change and merge but this way I can check each df as it comes
    # For each child location:
    #   1. make a copy of the parent_distribution df
    #   2. set the location id in the copy to the child location
    #   3. assert that dataframes are identical, besides the location_id
    #   4. append to the list of dataframes
    children_distribution_dfs_list = []
    for child_loc in children_location_ids:
        child_distribution_df = parent_distribution_df.copy()
        child_distribution_df.location_id = child_loc
        pd.testing.assert_frame_equal(left=child_distribution_df.drop("location_id", axis=1),
                                      right=parent_distribution_df.drop("location_id", axis=1),
                                      check_like=False)
        children_distribution_dfs_list.append(child_distribution_df)
    assert(len(children_distribution_dfs_list) == len(children_distribution_dfs_list))
    
    child_distribution_df = pd.concat(children_distribution_dfs_list, ignore_index=True)
    assert child_distribution_df.shape[0] > 0
    assert child_distribution_df.shape[0] == parent_distribution_df.shape[0] * len(children_distribution_dfs_list)
    assert set(child_distribution_df.location_id) == set(children_location_ids)
    assert not child_distribution_df.duplicated().any()
    return child_distribution_df

In [ ]:
phl_sub = make_weights_for_new_subnationals(distribution_df=old_distribution, parent_id=16)

In [ ]:
# make a child distribution df for each of the new parent_ids
children_distribution_dfs_list = []
for parent_id in new_parent_location_ids:
    children_distribution_dfs_list.append(
        make_weights_for_new_subnationals(distribution_df=old_distribution, parent_id=parent_id)
    )
assert len(children_distribution_dfs_list) == len(new_parent_location_ids)

In [ ]:
all_new_subnational_locations_dist_df = pd.concat(
    children_distribution_dfs_list, ignore_index=True
)

In [ ]:
all_new_subnational_locations_dist_df.shape

In [ ]:
old_distribution.shape

In [ ]:
new_distribution = pd.concat(
    [old_distribution, all_new_subnational_locations_dist_df], ignore_index=True
)

In [ ]:
check_missing_locations(df=new_distribution, new_location_ids=new_location_ids)

In [ ]:
location_metadata

In [ ]:
def print_location_information(location):
    
    location_name = new_location_metadata[
        new_location_metadata.location_id == location
    ].location_name.unique()[0]
    
    parent_id = new_location_metadata[
        new_location_metadata.location_id == location
    ].parent_id.unique()[0]
    
    parent_location_name = new_location_metadata[
        new_location_metadata.location_id == parent_id
    ].location_name.unique()[0]
    
    print("Location: {}".format(location))
    print("location_name: {}".format(location_name))
    print("parent_id: {}".format(parent_id))
    print("parent_id location_name: {}".format(parent_location_name))
    print("\n")

In [ ]:
for l in [320, 416, 393, 396, 367, 369, 374, 380, 413]:
    print_location_information(l)

In [ ]:
old_distribution[old_distribution.location_id == 104]

In [ ]:
oceania_locs = old_location_metadata[old_location_metadata.parent_id == 21].location_id.unique().tolist()

In [ ]:
pop = db_queries.get_population(gbd_round_id=6, location_id=oceania_locs, age_group_id=22, sex_id=3, year_id=2015)

In [ ]:
pop = pop.merge(new_location_metadata[['location_id', 'location_name']], how='left', on='location_id')

In [ ]:
pop.sort_values("population", ascending=False)

Lets make a dictionary mapping new_location to reference location

In [ ]:
new_locations_only_metadata[
        new_locations_only_metadata.parent_id == 21
    ].location_id.unique().tolist()

There are missing locations

[320, 416, 393, 396, 367, 369, 374, 380, 413]

In [ ]:
set([320, 369, 374, 380, 413, 416]).symmetric_difference(set([320, 416, 393, 396, 367, 369, 374, 380, 413]))

In [ ]:
for l in [367, 393, 396]:
    print_location_information(l)

In [ ]:
location_metadata.query("location_name == 'Antigua and Barbuda'")

In [ ]:
dist_new_to_old_loc_map = {
    # Oceania locations use Paupa New Guinea
    320: 26,
    369: 26,
    374: 26,
    380: 26,
    413: 26,
    416: 26,
    # for San Marino use Italy
    396: 86,
    # for Monaco use France
    367: 80,
    # for Saint Kitts and Nevis use Antigua and Barbuda
    393: 105
}

In [ ]:
# check mapping
for key in dist_new_to_old_loc_map:
    new_location_name = new_location_metadata[
        new_location_metadata.location_id == key
    ].location_name.unique()[0]
    
    old_location_name = new_location_metadata[
        new_location_metadata.location_id == dist_new_to_old_loc_map[key]
    ].location_name.unique()[0]
    
    print("{} : {}".format(location_name, old_location_name))

In [ ]:
def make_weights_for_new_locations(distribution_df, new_location_id, dist_new_to_old_loc_map):
    
    reference_location_id = dist_new_to_old_loc_map[new_location_id]
    
    new_location_name = new_location_metadata[
        new_location_metadata.location_id == new_location_id
    ].location_name.unique()[0]
    reference_location_name = new_location_metadata[
        new_location_metadata.location_id == reference_location_id
    ].location_name.unique()[0]
    
    print("Giving {} the distribution of {}".format(new_location_name, reference_location_name))
    print("Giving {} the distribution of {}".format(new_location_id, reference_location_id))
    print("\n")
    
    # subset distribution df to just the dist for the reference location
    reference_distribution_df = distribution_df[distribution_df.location_id == reference_location_id].copy()
    assert reference_distribution_df.shape[0] > 0, "NO DATA FOR THIS LOCATION: {}".foramt(reference_location_id)
    
    # replace location_id in reference_distribution_df with the location id of the new location
    reference_distribution_df.location_id = new_location_id
    print(reference_distribution_df.shape)
    return reference_distribution_df

In [ ]:
len(dist_new_to_old_loc_map)

In [ ]:
new_locations_distribution_df_list = []
for key in dist_new_to_old_loc_map:
    new_locations_distribution_df_list.append(
        make_weights_for_new_locations(old_distribution, key, dist_new_to_old_loc_map)
    )
assert len(new_locations_distribution_df_list) == len(dist_new_to_old_loc_map)

In [ ]:
new_locations_dist_df = pd.concat(new_locations_distribution_df_list, ignore_index=True)

In [ ]:
new_distribution = pd.concat([old_distribution,
                              new_locations_dist_df,
                              all_new_subnational_locations_dist_df], ignore_index=True)

In [ ]:
new_distribution.shape

In [ ]:
old_distribution.shape

In [ ]:
check_missing_locations(df=new_distribution, new_location_ids=new_location_ids)

In [ ]:
new_distribution.head()

In [ ]:
assert not new_distribution.duplicated().any()

In [ ]:
assert not new_distribution.drop("weight", axis=1).duplicated().any()

In [ ]:
new_location_metadata = db_queries.get_location_metadata(gbd_round_id=6, location_set_id=82)
old_location_metadata = db_queries.get_location_metadata(gbd_round_id=5, location_set_id=82)
new_location_ids = list(set(new_location_metadata.location_id) - set(old_location_metadata.location_id))


In [ ]:
assert set(new_location_ids) < set(new_distribution.location_id.unique())

In [ ]:
new_distribution.to_csv(
    ,
    index=False
)